In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
look_back = 600   # Number of previous timesteps to observe for each value

myOptimiser = tf.keras.optimizers.Adam(1e-4)

# Create data
dt = tf.constant(0.2)
total_time = look_back * dt  # Time at end of simulation (seconds)

noiseI = 0.0        # Set this to non-zero if you want to add noise to current values (A)
noiseV = 0.0        # Set this to non-zero if you want to add noise to voltage values (V)

I_max = tf.constant(-3.6 * 8.0)     # Maximum value for the current. Recommend setting this using C-rate (Capacity * C-rate). 18650 cell has capacity of roughly 3600mAh
I_min = tf.constant(3.6 * 1.0)

# Create time variables
time = np.linspace(0.0, total_time, look_back, endpoint=True, dtype=np.float32)

# Define electrical parameters
V_ocv = tf.constant(3.75)
V_ub = tf.constant(4.0)
V_lb = tf.constant(3.5)

R0 = tf.constant(0.00296)
R1 = tf.constant(0.00152)
R2 = tf.constant(4.48728)

C1 = tf.constant(13520.8)
C2 = tf.constant(242560.0)

Q_nominal = tf.constant(3.6*3600)

print(f'Time simulated: {total_time} s')

In [ ]:
def create_data(I_thisBatch, SOC_t0, pulse_length, pause_length):
    # Choose random current (Can be positive or negative with a maximum magnitude of I_max set in previous section)
    pulse_start = 0
    period_length = int((pulse_length + pause_length)/dt)
    pulse_end = pulse_start + int(pulse_length/dt)
    period_end = period_length

    # Initialise arrays
    I = np.zeros(look_back, dtype=np.float32)

    I[pulse_start:pulse_end] = I_thisBatch      # Leave first three timesteps as zero, then the first half of time is the randomised current

    while pulse_start+period_length-1 <= look_back:
        I[pulse_start:pulse_end] = I_thisBatch      # Leave first three timesteps as zero, then the first half of time is the randomised current
        I[pulse_end+1:period_end] = 0.0             # Second half of the time is zero to show voltage relaxation

        pulse_start = period_end + 1
        pulse_end = pulse_start + int(pulse_length/dt)
        period_end = period_end + period_length

    I[0] = 0.0

    # Set current and voltage across each branched resistor to zero
    I1 = np.zeros(look_back, dtype=np.float32)
    I2 = np.zeros(look_back, dtype=np.float32)

    V = np.zeros(look_back, dtype=np.float32)
    V0 = np.zeros(look_back, dtype=np.float32)
    V1 = np.zeros(look_back, dtype=np.float32)
    V2 = np.zeros(look_back, dtype=np.float32)

    SOC = np.zeros(look_back, dtype=np.float32)
    SOC[0] = SOC_t0

    V[0] = V_ocv
    V0[0] = 0.0
    V1[0] = 0.0
    V2[0] = 0.0

    # Calculate values at each timestep
    V0 = I * R0   # V0 is instantaneous response so not reliant on previous timestep

    for t in range(1,look_back,1):
        SOC[t] = SOC[t-1] + (I[t]*dt)/Q_nominal

        #R1 = np.sin(SOC[t]) + 0.01*SOC[t]**2 + 0.01*SOC[t]**3

        I1[t] = (tf.math.exp(-dt/(R1*C1)) * I1[t-1]) + ((1.0 - tf.math.exp(-dt/(R1*C1))) * I[t-1])
        I2[t] = (tf.math.exp(-dt/(R2*C2)) * I2[t-1]) + ((1.0 - tf.math.exp(-dt/(R2*C2))) * I[t-1])

        V1[t] = (I1[t] * R1)
        V2[t] = (I2[t] * R2)

        V[t] = V_ocv + V0[t] + V1[t] + V2[t]  # Total voltage across cell

    # Add desired noise level (Can be positive or negative)
    I += (np.random.rand(V.shape[0]) - 0.5) * noiseI
    V += (np.random.rand(V.shape[0]) - 0.5) * noiseV

    # Return data
    return(I, SOC, V0, V1, V2, V)

In [ ]:
def create_dataset(I_thisBatch, SOC_t0, pulse_length, pause_length):
    # Create data
    I, SOC, V0, V1, V2, V = create_data(I_thisBatch, SOC_t0, pulse_length, pause_length)

    # Assign data for training
    x_train = np.zeros((1, look_back, 2))
    x_train[0,:,0] = I
    x_train[0,:,1] = SOC[0]
    y_train = np.zeros((1, look_back, 2))
    y_train[0,:,0] = V
    y_train[0,:,1] = SOC

    return x_train, y_train

x, y_full = create_dataset(I_max, 0.5, 10.0, 10.0)

plt.plot(np.linspace(0.0, dt*look_back, num=look_back), x[0,:,0])
plt.show()
plt.close()

In [ ]:
# Custom loss

def custom_loss_wrapper(y_true, y_pred):

    def calculate_dSOC(I, SOC_pred):
        # dSOC at t=t+1
        dSOC_pred = SOC_pred[:,1:] - SOC_pred[:,:-1]
        # dSOC at t=t+1
        dSOC_theory = (I[:,1:]*dt)/Q_nominal
        dSOC_loss = tf.reduce_sum(tf.square(dSOC_theory - dSOC_pred))
        return dSOC_loss

    def calculate_V(I, yV, ySOC, SOC_t0):
        # Set current and voltage across each branched resistor to zero
        V_theory = tf.zeros((tf.shape(I)[0], 1)) + V_ocv

        #SOC = tf.ones((tf.shape(I)[0], 1)) * SOC_t0
        SOC = tf.expand_dims(SOC_t0, axis=-1)

        # Calculate values at each timestep
        V0 = I * R0   # V0 is instantaneous response so not reliant on previous timestep

        I1 = tf.zeros_like(I[:,0])
        I2 = tf.zeros_like(I[:,0])

        for t in range(1,look_back):
            SOC = tf.concat(values=[SOC, tf.expand_dims(SOC[:,-1] + (I[:,t]*dt)/Q_nominal, axis=1)], axis=1)

            I1 = (tf.math.exp(-dt/(R1*C1)) * I1) + ((1.0 - tf.math.exp(-dt/(R1*C1))) * I[:,t-1])
            I2 = (tf.math.exp(-dt/(R2*C2)) * I2) + ((1.0 - tf.math.exp(-dt/(R2*C2))) * I[:,t-1])

            V1 = (I1 * R1)
            V2 = (I2 * R2)

            V_theory = tf.concat([V_theory, tf.expand_dims(V_ocv + V0[:,t] + V1 + V2, axis=1)], axis=1)  # Total voltage across cell

        dV_theory = V_theory[:,1:] - V_theory[:,:-1]
        dV_pred   = yV[:,1:] - yV[:,:-1]

        # Theory values start with initial conditions so are one timestep behind predicted values
        SOC_loss = tf.reduce_mean(tf.square(SOC[:,1:] - ySOC[:,:-1]))
        V_loss = tf.reduce_mean(tf.square(V_theory[:,1:] - yV[:,:-1]))
        dV_loss = tf.reduce_sum(tf.square((dV_theory - dV_pred)))
        return V_loss, dV_loss, SOC_loss





    def custom_loss(y_true, y_pred):
        # Gather inputs
        xI = y_true[:,:,0]
        yV = y_pred[:,:,0]
        ySOC = y_pred[:,:,1]
        SOC_t0 = y_true[:,0,1]

        loss = 0.0
        num_batches = tf.shape(y_true)[0]
        nt = tf.shape(y_true)[1]

        # SOC change loss
        dSOC_loss = calculate_dSOC(xI, ySOC)

        # Calculate voltage losses using ECM
        V_loss, dV_loss, SOC_loss = calculate_V(xI, yV, ySOC, SOC_t0)

        # Initial conditions
        t0_loss = tf.reduce_mean(tf.square(V_ocv - yV[:,0]))
        t0_loss += tf.reduce_mean(10.0*tf.square(SOC_t0[:] - ySOC[:,0]))

        loss = tf.reduce_sum([t0_loss, V_loss, SOC_loss, 100.0*dV_loss, 100.0*dSOC_loss])

        return loss

    return custom_loss(y_true, y_pred)

In [ ]:
def input_transform(x):
    return 2.0 * ((x - I_min) / (I_max - I_min)) - 1.0

def soc_transform(x):
    return (x - 0.5) * 2

def output_transform_V(x):
    return 0.5 * (x + 1.0) * (V_ub - V_lb) + V_lb

def output_transform_SOC(x):
    return 0.5 * (x + 1.0)

In [ ]:
from tensorflow.keras.layers import TimeDistributed as TD

# Input SOC at time t=0
SOC_input = tf.keras.Input(shape=(None, 1), name="soc")

# Normalise SOC input between -1 and 1
SOC_norm = tf.keras.layers.Activation(activation=soc_transform)(SOC_input)

# Current input at time t=t
i_input = tf.keras.Input(shape=(None, 1), name="current")

# Normalise current input between -1 and 1
i_norm = tf.keras.layers.Activation(activation=input_transform)(i_input)

# Combine inputs into one layer
model_norm_inp = tf.keras.layers.Concatenate(axis=-1)([i_norm, SOC_norm])

# FFN as encoder to RNN
model_norm = tf.keras.layers.Dense(units=32, activation="tanh")(model_norm_inp)
model_norm = tf.keras.layers.Dense(units=32, activation="tanh")(model_norm)

# RNN
model_x = tf.keras.layers.LSTM(units=32, activation="tanh", return_sequences=True)(model_norm)
model_x = tf.keras.layers.Add()([model_x, model_norm])
model_x = tf.keras.layers.LSTM(units=32, activation="tanh", return_sequences=True)(model_x) #(tf.concat([model_x, model_norm], axis=-1))

# FNN as decoder of RNN
model_x = tf.keras.layers.Add()([model_x, model_norm])
model_x = tf.keras.layers.Dense(units=32, activation="tanh")(model_x)
model_x = tf.keras.layers.Dense(units=32, activation="tanh")(model_x)
model_x = tf.keras.layers.Dense(units=2, activation="tanh")(model_x)

# Denormalise outputs to correct values
V_out_norm = tf.keras.layers.Activation(output_transform_V)(model_x[:,:,0:1]) #(tf.expand_dims(model_x[:,:,0], axis=-1))
SOC_out_norm = tf.keras.layers.Activation(output_transform_SOC)(model_x[:,:,1:2]) #(tf.expand_dims(model_x[:,:,1], axis=-1))

# Output V and SOC prediction at time t=t
model_output = tf.keras.layers.Concatenate(axis=-1)([V_out_norm, SOC_out_norm])

# Define and compile model
model = tf.keras.Model(inputs=[i_input, SOC_input], outputs=model_output)
model.compile(loss=custom_loss_wrapper, optimizer=myOptimiser)

model.summary()
loss_train = []
mse_V_test = []
mse_SOC_test = []

In [ ]:
def create_multi_data(I_thisBatch, SOC_t0, pulse_length, pause_length):
    # Initialise arrays
    N = I_thisBatch.shape[0]
    I = np.zeros((N, look_back), dtype=np.float32)

    for ii in range(pause_length.shape[0]):
        pulse_start = 0
        period_length = int((pulse_length[ii] + pause_length[ii])/dt)
        pulse_end = pulse_start + int(pulse_length[ii]/dt)
        period_end = period_length

        I[ii, pulse_start:pulse_end] = I_thisBatch[ii]

        while pulse_start+period_length-1 <= look_back:
            I[ii, pulse_start:pulse_end] = I_thisBatch[ii]
            I[ii, pulse_end+1:period_end] = 0.0

            pulse_start = period_end + 1
            pulse_end = pulse_start + int(pulse_length[ii]/dt)
            period_end = period_end + period_length

    I[:, 0] = 0.0

    # Set current and voltage across each branched resistor to zero
    I1 = np.zeros((N, look_back), dtype=np.float32)
    I2 = np.zeros((N, look_back), dtype=np.float32)

    V = np.zeros((N, look_back), dtype=np.float32)
    V0 = np.zeros((N, look_back), dtype=np.float32)
    V1 = np.zeros((N, look_back), dtype=np.float32)
    V2 = np.zeros((N, look_back), dtype=np.float32)

    SOC = np.zeros((N, look_back), dtype=np.float32)
    SOC[:, 0] = SOC_t0

    V[:,0] = V_ocv
    V0[:,0] = 0.0
    V1[:,0] = 0.0
    V2[:,0] = 0.0

    # Calculate values at each timestep
    V0 = I * R0   # V0 is instantaneous response so not reliant on previous timestep

    for t in range(1,look_back,1):
        SOC[:,t] = SOC[:,t-1] + (I[:,t]*dt)/Q_nominal

        #R1 = np.sin(SOC[t]) + 0.01*SOC[t]**2 + 0.01*SOC[t]**3

        I1[:,t] = (tf.math.exp(-dt/(R1*C1)) * I1[:,t-1]) + ((1.0 - tf.math.exp(-dt/(R1*C1))) * I[:,t-1])
        I2[:,t] = (tf.math.exp(-dt/(R2*C2)) * I2[:,t-1]) + ((1.0 - tf.math.exp(-dt/(R2*C2))) * I[:,t-1])

        V1[:,t] = (I1[:,t] * R1)
        V2[:,t] = (I2[:,t] * R2)

        V[:,t] = V_ocv + V0[:,t] + V1[:,t] + V2[:,t]  # Total voltage across cell

    # Add desired noise level (Can be positive or negative)
    I += (np.random.rand(V.shape[0], V.shape[1]) - 0.5) * noiseI
    V += (np.random.rand(V.shape[0], V.shape[1]) - 0.5) * noiseV

    # Return data
    return(I, SOC, V0, V1, V2, V)

def create_multi_dataset(I_thisBatch, SOC_t0, pulse_length, pause_length):
    # Create data
    I, SOC, V0, V1, V2, V = create_multi_data(I_thisBatch, SOC_t0, pulse_length, pause_length)

    # Assign data for training
    x_train = np.zeros((I.shape[0], look_back, 2))
    x_train[:,:,0] = I
    for ii in range(SOC.shape[0]):
        x_train[ii,:,1] = SOC[ii,0]
    y_train = np.zeros((I.shape[0], look_back, 2))
    y_train[:,:,0] = V
    y_train[:,:,1] = SOC

    return x_train, y_train

In [ ]:
noiseI = 0.0        # Set this to non-zero if you want to add noise to current values (A)
noiseV = 0.0        # Set this to non-zero if you want to add noise to voltage values (V)

I_range     = np.array([-1, -0.5, -0.25, 0, 0.25, 0.5, 1, 2, 3, 4, 5, 6, 7, 8], dtype="float32") * -3.6
SOC_range   = np.linspace(0.0, 1.0, num=101, endpoint=True)
pulse_range = np.linspace(10.0, look_back, num=100)
pause_range = np.linspace(0.0, look_back/2.0, num=100)
nData = I_range.shape[0] * SOC_range.shape[0] * pulse_range.shape[0] * pause_range.shape[0]
print(f"{nData} possible data combinations")

I_array = np.random.choice(I_range, 200, replace=True)
SOC_array = np.random.choice(SOC_range, 200, replace=True)
pulse_array = np.random.choice(pulse_range, 200, replace=True)
pause_array = np.random.choice(pause_range, 200, replace=True)

del I_range, SOC_range, pulse_range, pause_range

x_train, y_train = create_multi_dataset(I_array, SOC_array, pulse_array, pause_array)

# Format as tensor and ignore first row to get correct amount of data points
x_train = tf.cast(x_train, dtype=tf.float32)
y_train = tf.cast(y_train, dtype=tf.float32)

In [ ]:
mse_V_test = []
mse_SOC_test = []
loss_train = []

In [ ]:
# Record learning process
Epochs = 5

pred = np.zeros((Epochs, look_back, 2), dtype=np.float32)
x_test, y_test = create_dataset(I_max, 0.5, 30.0, 30.0)

for ep in range(Epochs):
  print(f"Epoch {ep+1}/{Epochs}...")
  history = model.fit(x={"current" : x_train[:,:,0], "soc" : x_train[:,:,1]}, y=x_train, shuffle=True, epochs=5, batch_size=512, verbose=1)
  pred[ep,:,:] = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))[0,:,:]
  np.savetxt(f"GINN_Prediction_{ep+1}.csv", pred[ep,:,:], delimiter=",")
  mse_V_test = np.append(mse_V_test, tf.sqrt(tf.reduce_mean((tf.square(y_test[0,:,0] - pred[ep,:,0])/y_test[0,:,0]))))
  mse_SOC_test = np.append(mse_SOC_test, tf.sqrt(tf.reduce_mean((tf.square(y_test[0,:,1] - pred[ep,:,1])/y_test[0,:,1]))))
  loss_train = np.append(loss_train, history.history["loss"])
  print(f"  Loss: {np.float32(np.mean(history.history['loss']))}    V MSE: {mse_V_test[ep]}%    SoC MSE: {mse_SOC_test[ep]}%")

In [ ]:
# Plot training progress
plt.figure(figsize=(15,5))
plt.plot(loss_train)
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.yscale('log')
#plt.xscale('log')
plt.show()
plt.close()

np.savetxt("loss_GINN.csv", loss_train, delimiter=",")

plt.figure(figsize=(10,5))
plt.plot(mse_V_test)
plt.ylabel("MSE_{V}")
plt.show()
plt.close()

plt.figure(figsize=(10,5))
plt.plot(mse_SOC_test)
plt.ylabel("MSE_{SOC}")
plt.show()
plt.close()

# Test Model

In [ ]:
# Format prediction data
V_pred = pred[:,:,0]
SOC_pred = pred[:,:,1]

# Plot results from test data
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
x_test, y_test = create_dataset(I_max, 0.5, 30.0, 30.0)

#    Plot voltage
plt.figure(figsize=(10,15))
plt.subplot(4,1,1)
plt.plot(time_test, y_test[0,:,0], '--r', label='True')
for ii in range(Epochs):
  plt.plot(time_test, V_pred[ii,:], label=f'itter {ii}')
plt.legend()
plt.ylabel("Voltage (V)")
plt.xlabel("Time (s)")
plt.title("Predicted voltage response")

plt.subplot(4,1,2)
x_test, y_test = create_dataset(0.0, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,0], '--r', label='True')
plt.plot(time_test, V_test[0,:,0], 'k', label='Pred')
plt.legend()
plt.ylabel("Voltage (V)")
plt.xlabel("Time (s)")
plt.title("Predicted voltage response")

plt.subplot(4,1,3)
x_test, y_test = create_dataset(I_max, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,0], '--r', label='True')
plt.plot(time_test, V_test[0,:,0], 'k', label='Pred')
plt.legend()
plt.ylabel("Voltage (V)")
plt.xlabel("Time (s)")
plt.title("Predicted voltage response")

plt.subplot(4,1,4)
x_test, y_test = create_dataset(3.6, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,0], '--r', label='True')
plt.plot(time_test, V_test[0,:,0], 'k', label='Pred')
plt.legend()
plt.ylabel("Voltage (V)")
plt.xlabel("Time (s)")
plt.title("Predicted voltage response")

plt.show()
plt.close()

In [ ]:
#    Plot SOC
plt.figure(figsize=(10,15))
plt.subplot(4,1,1)
x_test, y_test = create_dataset(I_max, 0.5, 30.0, 30.0)
plt.plot(time_test, y_test[0,:,1], '--r', label='True')
for ii in range(Epochs):
  plt.plot(time_test, SOC_pred[ii,:], label=f'itter {ii}')
plt.legend()
plt.ylabel("SOC")
plt.xlabel("Time (s)")
plt.title("Predicted SOC response")

plt.subplot(4,1,2)
x_test, y_test = create_dataset(0.0, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,1], '--r', label='True')
plt.plot(time_test, V_test[0,:,1], 'k', label='Pred')
plt.legend()
plt.ylabel("SOC")
plt.xlabel("Time (s)")
plt.title("Predicted SOC response")

plt.subplot(4,1,3)
x_test, y_test = create_dataset(I_max, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,1], '--r', label='True')
plt.plot(time_test, V_test[0,:,1], 'k', label='Pred')
plt.legend()
plt.ylabel("SOC")
plt.xlabel("Time (s)")
plt.title("Predicted SOC response")

plt.subplot(4,1,4)
x_test, y_test = create_dataset(3.6, 0.5, 30.0, 30.0)
V_test = np.array(model.predict([x_test[:,:,0], x_test[:,:,1]]))
time_test = np.linspace(0.0, look_back*dt, num=look_back, endpoint=True)
plt.plot(time_test, y_test[0,:,1], '--r', label='True')
plt.plot(time_test, V_test[0,:,1], 'k', label='Pred')
plt.legend()
plt.ylabel("SOC")
plt.xlabel("Time (s)")
plt.title("Predicted SOC response")

plt.show()
plt.close()

# Test timing

In [ ]:
from time import time
import tracemalloc

nCells = 50
timers = np.zeros((nCells,2))
memory_current = np.zeros((nCells,2))
memory_peak = np.zeros((nCells,2))

for N in range(1,nCells):
    x_test = np.zeros((N, look_back, 2))

    for ii in range(2):
        x_test[:,:,0], x_test[:,:,1], V0_test, V1_test, V2_test, y_test = create_multi_data(tf.ones((N,))*I_max, tf.ones((N,))*0.5, tf.ones((N,))*30.0, tf.ones((N,))*30.0)

        tracemalloc.start()
        t_start = time()
        x_test[:,:,0], x_test[:,:,1],  V0_test, V1_test, V2_test, y_test = create_multi_data(tf.ones((N,))*I_max, tf.ones((N,))*0.5, tf.ones((N,))*30.0, tf.ones((N,))*30.0)
        t_ECM = time() - t_start
        mem_ECM = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        tracemalloc.start()
        t_start = time()
        V_test = model.predict([x_test[:,:,0], x_test[:,:,1]])
        t_PINN = time() - t_start
        mem_PINN = tracemalloc.get_traced_memory()
        tracemalloc.stop()

    print(f"For {N} cells and {x_test.shape[1]} timesteps")
    print(f"ECM:  {t_ECM}s")
    print(f"PINN: {t_PINN}s")

    timers[N,0] = t_ECM
    timers[N,1] = t_PINN

    memory_current[N,0] = mem_ECM[0]
    memory_current[N,1] = mem_PINN[0]
    memory_peak[N,0] = mem_ECM[1]
    memory_peak[N,1] = mem_PINN[1]

np.savetxt("timings.csv", timers, delimiter=",")
np.savetxt("memory_current.csv", memory_current, delimiter=",")
np.savetxt("memory_peak.csv", memory_peak, delimiter=",")

In [ ]:
# Plot time and memory requirements

plt.figure()
plt.plot(timers[:,0], color=[1, 0, 0], label="ECM")
plt.plot(timers[:,1], color=[0, 0, 1], label="GINN")
plt.xlabel("Number of cells being simulated simultaneously")
plt.ylabel("Time to simulate (s)")
plt.legend()
plt.show()
plt.close()

plt.figure()
plt.plot(memory_current[:,0], color=[1, 0.5, 0.5], label="Avg memory - ECM")
plt.plot(memory_current[:,1], color=[0, 0, 1], label="Avg memory - GINN")
plt.plot(memory_peak[:,0], "r--", label="Peak memory - ECM")
plt.plot(memory_peak[:,1], "--", label="Peak memory - GINN")
plt.xlabel("Number of cells being simulated simultaneously")
plt.ylabel("Time to simulate (s)")
plt.legend()
plt.show()
plt.close()

In [ ]:
model.save("GINN_ECM")
model.save("GINN_ECM.h5")

# To load existing model

In [ ]:
model = tf.keras.models.load_model("GINN_ECM.h5", custom_objects = {"custom_loss_wrapper" : custom_loss_wrapper,
                                                             "input_transform" : input_transform,
                                                             "soc_transform" : soc_transform,
                                                             "output_transform_V" : output_transform_V,
                                                             "output_transform_SOC" : output_transform_SOC
                                                             })

def input_transform(x):
    return 2.0 * ((x - I_min) / (I_max - I_min)) - 1.0

def soc_transform(x):
    return (x - 0.5) * 2

def output_transform_V(x):
    return 0.5 * (x + 1.0) * (V_ub - V_lb) + V_lb

def output_transform_SOC(x):
    return 0.5 * (x + 1.0)